In [2]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import stock_mapping as sm
import warnings
import asyncio
import twap
import time
from datetime import datetime, timedelta
from class_mapping import StockSymbolMapper
pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

if not mt5.initialize():
    print("initialize() failed")
else:
    print("MT5 successfully initialised.\n")

MT5 successfully initialised.



In [3]:
mt5.market_book_add('CIMB')
add = mt5.market_book_get('CIMB')
print(add)

(BookInfo(type=1, price=5.88, volume=27500, volume_dbl=27500.0), BookInfo(type=1, price=5.87, volume=38500, volume_dbl=38500.0), BookInfo(type=1, price=5.86, volume=85600, volume_dbl=85600.0), BookInfo(type=1, price=5.85, volume=123300, volume_dbl=123300.0), BookInfo(type=1, price=5.84, volume=74100, volume_dbl=74100.0), BookInfo(type=2, price=5.83, volume=100400, volume_dbl=100400.0), BookInfo(type=2, price=5.82, volume=78800, volume_dbl=78800.0), BookInfo(type=2, price=5.8100000000000005, volume=104500, volume_dbl=104500.0), BookInfo(type=2, price=5.8, volume=181500, volume_dbl=181500.0), BookInfo(type=2, price=5.79, volume=107400, volume_dbl=107400.0))


In [ ]:
yf_symbols =['5227.KL','5176.KL','5212.KL','5106.KL','5110.KL','5307.KL',
            '6947.KL','6012.KL','1155.KL','1295.KL','1066.KL','1015.KL','6888.KL',
            '1818.KL','2488.KL','5185.KL','5258.KL','6139.KL','9296.KL','1082.KL',
            '6459.KL','3034.KL','5273.KL','4731.KL','7172.KL','5284.KL','9822.KL',
            '3476.KL','7100.KL','2852.KL','7095.KL','4758.KL','5302.KL','0151.KL',
            '7233.KL','5271.KL','5916.KL','0215.KL','1368.KL','5000.KL','7231.KL',
            '7034.KL','8907.KL','6556.KL','7245.KL','6971.KL','0223.KL','7195.KL',
            '8052.KL','7609.KL','7222.KL','5673.KL','7229.KL','0099.KL','7133.KL',
            '5568.KL','5192.KL','7029.KL','7197.KL','7004.KL','3816.KL','5014.KL',
            '5246.KL','6521.KL','2062.KL','7293.KL','5199.KL','5141.KL','3042.KL',
            '5071.KL','4324.KL','5132.KL','5249.KL','5200.KL','8206.KL','5401.KL',
            '3158.KL','5053.KL','5236.KL','5038.KL','6114.KL','7249.KL','3182.KL',
            '4715.KL','5296.KL','7084.KL','4588.KL','4006.KL','5248.KL','3301.KL',
            '4162.KL','1619.KL','5102.KL','7052.KL','5306.KL','8532.KL','3859.KL',
            '6599.KL','5983.KL','5517.KL','7237.KL','0186.KL','5202.KL','7087.KL',
            '5908.KL','9172.KL','7216.KL','7103.KL','6432.KL','7252.KL','9288.KL',
            '7107.KL','7085.KL','7230.KL','0097.KL','0208.KL','5005.KL','0128.KL',
            '7204.KL','5292.KL','7160.KL','5301.KL','5309.KL','5286.KL','0146.KL',
            '7022.KL','5347.KL','6742.KL','4677.KL','5209.KL','3069.KL','5168.KL',
            '5878.KL','7153.KL','7090.KL','7148.KL','0001.KL','1961.KL','5222.KL',
            '2291.KL','5254.KL','5126.KL','5027.KL','5138.KL','5012.KL','9059.KL',
            '6262.KL','3336.KL','5211.KL','7161.KL','5293.KL']
file_path = r'C:\Users\jeffnsy\Desktop\Python codes\midas_touch2\stock_mapping.xlsx'
mt5_symbol = sm.get_ticker_list(yf_symbols, file_path=file_path)
print(mt5_symbol)

In [ ]:
yf_symbols = ["1082.KL","1295.KL","5306.KL","1155.KL","5202.KL","5168.KL"]#'7195.KL',
symbols = [symbol + "SE" for symbol in yf_symbols]
api_token = "65090d11a67b45.83965572"
data = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(data)
#rename columns
mapping_file_path = "C:/Users/jeffnsy/Desktop/Python codes/midas_touch2/stock_mapping.xlsx"
mt5_symbol = sm.get_ticker_list(yf_symbols, file_path=mapping_file_path)
price_df.columns = mt5_symbol
#price_df.columns = yf_symbols
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)
price_df.tail()

In [ ]:
###dummy weights to test
import random
def generate_random_series(tickers, seed=None):
    if seed is not None:
        random.seed(seed)
    random_values = [random.random() for _ in tickers]
    total = sum(random_values)
    normalized_values = [value / total for value in random_values]
    return pd.Series(normalized_values, index=tickers)
latest_weights = generate_random_series(mt5_symbol)
latest_weights

In [ ]:
total_investment = 100000 #TRY 15K 10K 5K
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')
if previous_allocation_df is not None and previous_portfolio_df is not None:
    # Process the existing CSV data
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    # Handle the initial run scenario
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1
)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1
)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)

for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]

########################---async start here---#########################
print(allocation_df)
order_ids = mt2.execute_trades_from_data(allocation_df)
time.sleep(5)
orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]
print(f'orders_list: {orders_list}')
deleted_volumes = mt2.delete_orders(orders_list)
print(f'deleted_volumes: {deleted_volumes}')
########################---end here----#################################
for order in deleted_volumes:
    symbol = order['symbol']
    volume_filled = order['volume_filled']
    if order['direction'] == mt5.ORDER_TYPE_BUY_LIMIT:
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Entry Units'] = volume_filled
    else: 
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Exit Units'] = volume_filled

    #allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Allocated Units'] = volume_filled
# async def execute_and_monitor(allocation_df, timeout=60):
#     print(allocation_df)
#     order_ids = mt2.execute_trades_from_data(allocation_df)
#     print(f'Waiting for {timeout/60} minutes..')
#     await asyncio.sleep(timeout)
#     orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]  # Renamed from orders_list
#     print(f'orders_list: {orders_list}')
#     resubmitted_orders = mt2.delete_and_resubmit_orders(orders_list)  # Renamed from resubmitted_orders
#     print(f'resubmitted_orders: {resubmitted_orders}')
#     return orders_list, resubmitted_orders  # Return the renamed variables

# # Execute the async function and capture the returned values with the new variable names
# orders_list, resubmitted_orders = asyncio.run(execute_and_monitor(allocation_df, timeout=90))

# Calculate the portfolio value
entry_unit = allocation_df['Entry Units']
exit_unit = allocation_df['Exit Units']
share_price = allocation_df['Share Price']
allocated_unit = allocation_df['Allocated Units']
if previous_allocation_df is not None and previous_portfolio_df is not None: #2nd run onwards
    starting_cash = previous_portfolio_df['Cash Value'].iloc[-1]
    current_share_value = ((entry_unit*share_price)-(exit_unit*share_price)).sum()
    current_share_value += previous_portfolio_df['Portfolio Value'].iloc[-1]

else: #first run
    starting_cash = total_investment
    current_share_value = (entry_unit*share_price).sum()

portfolio_df = mt2.compile_portfolio_data(allocation_df, starting_cash, current_share_value)

mt2.save_df_to_csv(portfolio_df, 
                   folder_name='daily_portfolio', 
                   file_name='balance',
                   append=True)
mt2.save_df_to_csv(allocation_df, 
                   folder_name='daily_allocation', 
                   file_name='allocation')


In [ ]:
deleted_volumes

In [ ]:
print(f'starting_cash: {starting_cash}')
print(f'current_share_value: {current_share_value}')

In [ ]:
portfolio_df

In [ ]:
allocation_df

In [ ]:
price_df